In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from allPrompts import load_prompts

df = load_prompts("../assets/conversations.json")
print(df.head())

# Example: your conversations dataframe
# df_titles = pd.DataFrame({'title': [convo['title'] for convo in data]})
titles = df['title'].dropna().tolist()  # if df contains your conversation titles

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(titles, show_progress_bar=True)